# FY2045 Quantum Mechanics - Numerical Exercise
Marte K. Høiskar and Maren Lium

## Problem 1

**Task:**
Calculate initial values for $\Psi _I (x, t)$ and $\Psi _R(x, t+\frac{\Delta t}{2})$ using Eq. (8). Normalise the wave function such that

\begin{equation}
\int_{0}^{L} |\Psi (x, t)|^2 \text{d}x = 1.
\end{equation}

Make a plot of the real and imaginary parts of Ψ, selecting suitable values for ∆x,
xs and σx to make your figure look similar to Figure 1. Make sure ∆x is small
enough that each oscillation of the wave function is resolved by enough points to
look fairly smooth. Furthermore, make a plot of the probability density for finding
the particle represented by the wave packet, given by $|Ψ(x, t)|^2$.

#### Solution:
Expanding Eq. (8) produce one real and one imaginary part,

\begin{equation}
\Psi (x, t) = Ce^{\frac{(x-x_s)^2}{2{\sigma_x}^2}}e^{i(k_0x-\omega t)} = Ce^{\frac{(x-x_s)^2}{2{\sigma_x}^2}}(\cos(k_0x-\omega t)+i\sin(k_0x-\omega t)).
\end{equation}

Hence, 

\begin{equation}
\Psi_I (x, t) = Ce^{\frac{(x-x_s)^2}{2{\sigma_x}^2}}\sin(k_0x-\omega t))
\quad\text{and}\quad
\Psi_R \left(x, t + \frac{\Delta t}{2}\right) = Ce^{\frac{(x-x_s)^2}{2{\sigma_x}^2}}\cos\left(k_0x-\omega \left(t+\frac{\Delta t}{2}\right)\right).
\end{equation}

The initial values for $\Psi _I (x, t)$ and $\Psi _R(x, t+\frac{\Delta t}{2})$ are

\begin{equation}
\Psi_I (x, 0) = Ce^{\frac{(x-x_s)^2}{2{\sigma_x}^2}}\sin(k_0x)
\quad\text{and}\quad
\Psi_R \left(x, \frac{\Delta t}{2}\right) = Ce^{\frac{(x-x_s)^2}{2{\sigma_x}^2}}\cos\left(k_0x-\omega \frac{\Delta t}{2}\right).
\end{equation}

## Problem 2

#### Task:
Start out a wave packet at $x_s = 5$, and propagate it a distance $\frac{L}{2}=10$, i.e., until it reaches $x = 15$. This will take a time $T = \frac{L}{2v_g}$, where $v_g$, given by 

\begin{equation}
v_g = \frac{\partial\omega}{\partial k}
\end{equation}

is the group velocity at the wave number k0. Repeat this procedure for different values of σx, for example $\sigma_x = 0.5$, $\sigma_x = 1.0$ and $\sigma_x = 2.0$. Describe what happens,and produce a figure that shows the initial probability density $|\Psi (x, t)|^2$ centered on $x_s = 5$, and the same for the propagated wave centered on $x = 15$, for two
different $\sigma_x$, for example $\sigma_x$ = 1.5 and $\sigma_x$ = 0.5.

To successfully carry out this computation, you will need to choose ∆t as described in the section on numerical stability. Begin by choosing a very small number, for example 100 times smaller than the limit. Propagate the wave function a few timesteps, and plot the results. See if you can increase $\Delta t$ somewhat, as that will make the computations go faster. Describe what happens if you increase $\Delta t$ too much. For the rest of the problems, choose a value for $\Delta t$ which is somewhat smaller than the largest value you found to work.

Note that the computations required for this problem should take no more than a few seconds of computer time.

#### Solution:

## Problem 3

## Problem 4

## Problem 5